La tabla "tbl_flujo_reporte" debe resumir en cada "id_item" el contenido del campo NETO de la tabla "tbl_transacciones" según "ITEM_COD", "ANIO" y "MES"

Evaluar las siguientes opciones:
1. ir creando una columna en tabla "tbl_flujo_reporte" para resumir la información de cada mes/año
2. Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD",   "ANIO" y "MES"

========================================================================================

1. **Crear una columna en la tabla "tbl_flujo_reporte" para resumir la información de cada mes/año**: 
* Esta opción implica agregar una nueva columna para cada mes/año en la tabla "tbl_flujo_reporte". Cada vez que se ingresen nuevos datos en la tabla "tbl_transacciones", tendrás que actualizar las columnas correspondientes en "tbl_flujo_reporte". 
* Esto podría ser eficiente si necesitas acceder rápidamente a los totales mensuales/año sin tener que realizar cálculos complejos en tiempo de consulta. 
* Sin embargo, podría volverse complicado si hay muchos meses/años diferentes o si la cantidad de datos es grande, ya que tendrías muchas columnas en la tabla "tbl_flujo_reporte".

2. **Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD", "ANIO" y "MES"**: 
* En esta opción, en lugar de tener una columna para cada mes/año en "tbl_flujo_reporte", tendrías una fila por cada combinación de "ITEM_COD", "ANIO" y "MES", con una columna que almacena el total NETO correspondiente. 
* Esto sería más escalable y flexible, ya que no tendrías que modificar la estructura de la tabla cada vez que agregues un nuevo mes/año. 
* Además, permitiría consultas más flexibles y eficientes sobre los datos agregados.

En general, la opción 2 parece ser la más flexible y escalable, ya que te permite almacenar los totales NETO de manera estructurada y realizar consultas eficientes sobre ellos. Sin embargo, la elección final dependerá de tus necesidades específicas y de la complejidad de tu aplicación.

========================================================================================

Uno de los propósitos de la tabla "tbl_flujo_reporte" es que sirva como puente para tomar datos que alimentarán un reporte

* Si el propósito principal de la tabla "tbl_flujo_reporte" es alimentar un reporte, entonces la opción 2 parece ser la más adecuada. 
* Al tener una fila por cada combinación de "ITEM_COD", "ANIO" y "MES" con el total NETO correspondiente, puedes generar fácilmente los datos necesarios para el reporte utilizando consultas SQL simples.
* Además, al utilizar esta estructura, la tabla "tbl_flujo_reporte" actuará como un puente eficiente para obtener los datos agregados necesarios para el reporte, ya que los totales NETO estarán precalculados y disponibles en la tabla, lo que evitará la necesidad de realizar cálculos complejos en tiempo de consulta.

Por lo tanto, la opción 2 te proporcionará una estructura eficiente y escalable para alimentar el reporte, lo que facilitará el proceso de generación del mismo y garantizará un rendimiento óptimo, especialmente si estás trabajando con grandes volúmenes de datos.

========================================================================================

In [4]:
import pandas as pd
import mysql.connector
import configparser

In [5]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_saldo_mes según rango definido en parámetros de entrada
# ============================================================================================================

def saldo_mes(anio_inicio, mes_inicio, anio_fin, mes_fin):

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_saldo_mes
    query = f"""
        SELECT *
        FROM tbl_saldo_mes
        WHERE (anio > {anio_inicio} OR (anio = {anio_inicio} AND mes >= {mes_inicio}))
        AND (anio < {anio_fin} OR (anio = {anio_fin} AND mes <= {mes_fin}))
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    saldos_mes = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return saldos_mes

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_saldo_mes según rango definido en parámetros de entrada

# Ejemplo de uso:
inf_saldos = saldo_mes(2023, 1, 2023, 2)
# print(reporte_mes)

# Crea la columna 'periodo' concatenando las columnas 'anio', 'mes'
inf_saldos['periodo'] = pd.to_datetime(inf_saldos['anio'].astype(str) + '-' + inf_saldos['mes'].astype(str) )
inf_saldos.tail()

C:\Users\gusta\AppData\Local\Temp\ipykernel_2480\268230371.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saldos_mes = pd.read_sql(query, conexion)


,id_saldo_mes,fecha,mes,anio,saldo_inicial,neto_mes,saldo_final,periodo
0,30,2023-01-31,1,2023,48779871.01,15151744.01,63931615.02,2023-01-01
1,31,2023-02-28,2,2023,63931615.02,17049613.28,80981228.30,2023-02-01


In [6]:
inf_saldos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_saldo_mes   2 non-null      int64         
 1   fecha          2 non-null      object        
 2   mes            2 non-null      int64         
 3   anio           2 non-null      int64         
 4   saldo_inicial  2 non-null      float64       
 5   neto_mes       2 non-null      float64       
 6   saldo_final    2 non-null      float64       
 7   periodo        2 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 256.0+ bytes


In [7]:
# ============================================================================================================
# Crea un dataframe resumen de inf_saldos
# con la suma de 'total_neto' por 'id_item_cod' y 'periodo'
# ============================================================================================================

# Crear el DataFrame reporte_final utilizando pivot_table
saldos_final = pd.pivot_table(inf_saldos, 
                               values='saldo_inicial', 
                               columns='periodo', 
                               aggfunc='sum')

# Conservar el formato datetime en los títulos de las columnas
saldos_final.columns = saldos_final.columns.strftime('%Y-%m')

saldos_final.reset_index(inplace = True)

# Mostrando el nuevo dataframe
saldos_final

periodo,index,2023-01,2023-02
0,saldo_inicial,48779871.01,63931615.02


In [9]:
saldos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    1 non-null      object 
 1   2023-01  1 non-null      float64
 2   2023-02  1 non-null      float64
dtypes: float64(2), object(1)
memory usage: 152.0+ bytes


In [10]:
# ============================================================================================================
# Renombrar la columna 'index' de la tabla saldos_final a 'id_item_cod'
# ============================================================================================================
saldos_final.rename(columns={'index':'id_item_cod'}, inplace=True)
saldos_final

periodo,id_item_cod,2023-01,2023-02
0,saldo_inicial,48779871.01,63931615.02


In [11]:
saldos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  1 non-null      object 
 1   2023-01      1 non-null      float64
 2   2023-02      1 non-null      float64
dtypes: float64(2), object(1)
memory usage: 152.0+ bytes


In [14]:
# ============================================================================================================
# Cambiar el contenido del registro en la columna 'id_item_cod' de la tabla saldos_final
# ============================================================================================================
saldos_final['id_item_cod'] = saldos_final['id_item_cod'].replace('saldo_inicial', '48')
saldos_final['id_item_cod'] = saldos_final['id_item_cod'].astype(int)

saldos_final

periodo,id_item_cod,2023-01,2023-02
0,48,48779871.01,63931615.02
